In [1]:
import pathlib
import sys
import time

sys.path.append("../featurization")
import itertools
from typing import Dict

import numpy as np
import pandas as pd
import scipy
import skimage
from colocalization import (
    measure_3D_colocalization,
    prepare_two_images_for_colocalization,
)
from data_writer import organize_featurization_data
from loading_classes import ImageSetLoader, ObjectLoader
from two_object_loading_classes import TwoObjectLoader

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [3]:
channel_mapping = {
    "nuclei": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "nuclei_mask": "nuclei_",
    "cell_mask": "cell_",
    "cytoplasm_mask": "cytoplasm_",
    "organoid_mask": "organoid_",
}

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

In [5]:
# get all channel combinations
channel_combinations = list(itertools.combinations(image_set_loader.image_names, 2))

In [6]:
output_list_of_dfs = []
for compartments in tqdm(
    image_set_loader.compartments, desc="Processing compartments", position=0
):
    for channel1, channel2 in tqdm(
        channel_combinations,
        desc="Processing channel combinations",
        leave=False,
        position=1,
    ):
        coloc_loader = TwoObjectLoader(
            image_set_loader=image_set_loader,
            compartment=compartments,
            channel1=channel1,
            channel2=channel2,
        )
        for object_id in tqdm(
            coloc_loader.object_ids,
            desc="Processing object IDs",
            leave=False,
            position=2,
        ):
            cropped_image1, cropped_image2 = prepare_two_images_for_colocalization(
                label_object1=coloc_loader.label_image,
                label_object2=coloc_loader.label_image,
                image_object1=coloc_loader.image1,
                image_object2=coloc_loader.image2,
                object_id1=object_id,
                object_id2=object_id,
            )
            colocalization_features = measure_3D_colocalization(
                cropped_image_1=cropped_image1,
                cropped_image_2=cropped_image2,
                thr=15,
                fast_costes="Accurate",
            )
            coloc_df = pd.DataFrame(colocalization_features, index=[0])
            coloc_df["object_id"] = object_id
            coloc_df["channel1"] = channel1
            coloc_df["channel2"] = channel2
            coloc_df["compartment"] = compartments
            output_list_of_dfs.append(coloc_df)

Processing compartments:   0%|          | 0/4 [00:00<?, ?it/s]

Processing channel combinations:   0%|          | 0/10 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/104 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/104 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/104 [00:00<?, ?it/s]

...

Processing object IDs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing object IDs:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
final_df = pd.concat(output_list_of_dfs, ignore_index=True)
final_df.head()

,Mean.Correlation.coeff,Median.Correlation.coeff,Min.Correlation.coeff,Max.Correlation.coeff,Mean.Manders.coeff.M1,Median.Manders.coeff.M1,Min.Manders.coeff.M1,Max.Manders.coeff.M1,Mean.Manders.coeff.M2,Median.Manders.coeff.M2,...,Min.Manders.Coeff.costes.M1,Max.Manders.Coeff.costes.M1,Mean.Manders.Coeff.costes.M2,Median.Manders.Coeff.costes.M2,Min.Manders.Coeff.costes.M2,Max.Manders.Coeff.costes..M2,object_id,channel1,channel2,compartment
0,-0.208877,-0.208877,-0.208877,-0.208877,0.249420,0.249420,0.249420,0.249420,0.428646,0.428646,...,0.228366,0.228366,1.0,1.0,1.0,1.0,2,nuclei,AGP,cell_mask
1,0.040319,0.040319,0.040319,0.040319,0.998825,0.998825,0.998825,0.998825,0.702443,0.702443,...,0.464290,0.464290,1.0,1.0,1.0,1.0,4,nuclei,AGP,cell_mask
2,-0.182645,-0.182645,-0.182645,-0.182645,0.214643,0.214643,0.214643,0.214643,0.740277,0.740277,...,0.145882,0.145882,1.0,1.0,1.0,1.0,7,nuclei,AGP,cell_mask
3,0.064261,0.064261,0.064261,0.064261,0.965160,0.965160,0.965160,0.965160,1.000000,1.000000,...,0.160226,0.160226,1.0,1.0,1.0,1.0,9,nuclei,AGP,cell_mask
4,-0.026820,-0.026820,-0.026820,-0.026820,0.900820,0.900820,0.900820,0.900820,0.982204,0.982204,...,0.322733,0.322733,1.0,1.0,1.0,1.0,12,nuclei,AGP,cell_mask
